In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import inspect_ai
from tqdm.notebook import tqdm
from huggingface_hub import login
import torch as t
import pynvml
import gc
import pandas as pd
from pprint import pprint
import os
from dotenv import load_dotenv

device = 'cuda' if t.cuda.is_available() else 'cpu'

In [2]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


### GPU Deets

In [28]:
import torch as t
import gc

free_memory, total_memory = t.cuda.mem_get_info()

# Convert bytes to GB
free_memory_gb = free_memory / (1024 * 1024 * 1024)
total_memory_gb = total_memory / (1024 * 1024 * 1024)
mem_used = t.cuda.device_memory_used() / (1024 ** 3)

print(f"Free GPU Memory: {free_memory_gb:.2f} GB")
print(f"Total GPU Memory: {total_memory_gb:.2f} GB")
print(f'Memory Used: {mem_used:.2f} GB')

Free GPU Memory: 37.30 GB
Total GPU Memory: 47.53 GB
Memory Used: 10.68 GB


In [24]:
t.cuda.get_device_properties()

_CudaDeviceProperties(name='NVIDIA RTX A6000', major=8, minor=6, total_memory=48669MB, multi_processor_count=84, uuid=fd7bc9d9-0e0c-1fff-6aeb-03f136700688, L2_cache_size=6MB)

In [25]:
print(t.cuda.memory_allocated() / 1024**2, "MB allocated")
print(t.cuda.memory_reserved() / 1024**2, "MB reserved")

0.0 MB allocated
0.0 MB reserved


In [26]:
# del model, tokenizer
t.cuda.empty_cache()
gc.collect()
# clean()

254